In [1]:
import pandas as pd

input_ = pd.read_excel('./5.vehicle_1.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_

,No,Item Number,Market,Page,JOIN_INP,Row,Year,Make,Model,Trim,Engine,Notes
0,1,196453166543,US,1,196453166543;1,1,2019,Ford,Explorer,Base Sport Utility 4-Door,2.3L 2300CC 140Cu. In. l4 GAS DOHC Turbocharged,Fit Turbocharged All Trims
1,1,196453166543,US,1,196453166543;1,2,2019,Ford,Explorer,Base Sport Utility 4-Door,3.5L 3496CC 213Cu. In. V6 GAS DOHC Naturally A...,Fit Explorer Naturally Aspirated - Front Wheel...
2,1,196453166543,US,1,196453166543;1,3,2019,Ford,Explorer,Limited Sport Utility 4-Door,2.3L 2300CC 140Cu. In. l4 GAS DOHC Turbocharged,Fit Turbocharged All Trims
3,1,196453166543,US,1,196453166543;1,4,2019,Ford,Explorer,Limited Sport Utility 4-Door,3.5L 3496CC 213Cu. In. V6 GAS DOHC Naturally A...,Fit Explorer Naturally Aspirated - Front Wheel...
4,1,196453166543,US,1,196453166543;1,5,2019,Ford,Explorer,Platinum Sport Utility 4-Door,3.5L 3496CC 213Cu. In. V6 GAS DOHC Turbocharged,Fit Turbocharged All Trims
...,...,...,...,...,...,...,...,...,...,...,...,...
41544,251,192737728468,US,6,192737728468;6,7,1992,Honda,Civic,DX Sedan 4-Door,1.5L 1493CC l4 GAS SOHC Naturally Aspirated,
41545,251,192737728468,US,6,192737728468;6,8,1992,Honda,Civic,EX Sedan 4-Door,1.6L 1588CC 98Cu. In. l4 GAS SOHC Naturally As...,
41546,251,192737728468,US,6,192737728468;6,9,1992,Honda,Civic,LX Sedan 4-Door,1.5L 1493CC l4 GAS SOHC Naturally Aspirated,
41547,251,192737728468,US,6,192737728468;6,10,1992,Honda,Civic,Si Hatchback 2-Door,1.6L 1588CC 98Cu. In. l4 GAS SOHC Naturally As...,


In [2]:
input_2 = input_[['Item Number']].drop_duplicates(keep='first', ignore_index=True)
input_2 = input_2.sort_values(by=['Item Number'],
                              ascending=[True],
                              ignore_index=True)

input_2

,Item Number
0,192731409825
1,192731591032
2,192732802929
3,192732835083
4,192733013576
...,...
246,196495713100
247,196495781333
248,196497159761
249,196497166909


In [3]:
from tqdm import tqdm

for i in tqdm(range(len(input_2)), desc='Progress', ncols=77):
    df_temp = input_[input_['Item Number'] == input_2.loc[i, 'Item Number']].copy().reset_index(drop=True)

    # = = = = = = = = = = = = = = =

    dict_vehicle = {}
    for j in range(len(df_temp)):
        if df_temp.loc[j, 'Market'] in ['US', 'UK', 'AU']:
            make, model, engine, year, year_2 = df_temp.loc[j, 'Make'], df_temp.loc[j, 'Model'], df_temp.loc[j, 'Engine'], int(df_temp.loc[j, 'Year']), int(df_temp.loc[j, 'Year'])
        elif df_temp.loc[j, 'Market'] == 'DE':
            make, model, engine, year, year_2 = df_temp.loc[j, 'Marke'], df_temp.loc[j, 'Modell'], df_temp.loc[j, 'Motor'], int(df_temp.loc[j, 'Baujahr'].split('-')[0].split('/')[0]), int(df_temp.loc[j, 'Baujahr'].split('-')[1].split('/')[0])

        if make not in dict_vehicle.keys():
            dict_vehicle[make] = {model: {engine: [year] if year == year_2 else [year, year_2]}}
        elif model not in dict_vehicle[make].keys():
            dict_vehicle[make][model] = {engine: [year] if year == year_2 else [year, year_2]}
        elif engine not in dict_vehicle[make][model].keys():
            dict_vehicle[make][model][engine] = [year] if year == year_2 else [year, year_2]
        else:
            if year not in dict_vehicle[make][model][engine]:
                dict_vehicle[make][model][engine].append(year)
            if year_2 not in dict_vehicle[make][model][engine]:
                dict_vehicle[make][model][engine].append(year_2)

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make in dict_vehicle.keys():
        for model in dict_vehicle[make].keys():
            list_year = []
            for engine in dict_vehicle[make][model].keys():
                list_year = list(set(list_year + dict_vehicle[make][model][engine]))
            year_begin = min(list_year)
            year_end = max(list_year)
            list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

    input_2.loc[i, 'Vehicle W/o Engine'] = '\n'.join(list_vehicle)

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make in dict_vehicle.keys():
        for model in dict_vehicle[make].keys():
            for engine in dict_vehicle[make][model].keys():
                year_begin = min(dict_vehicle[make][model][engine])
                year_end = max(dict_vehicle[make][model][engine])
                list_vehicle.append(f'{make} {model} {year_begin} {engine}'.strip() if year_begin == year_end else f'{make} {model} {year_begin}-{year_end} {engine}'.strip())

    input_2.loc[i, 'Vehicle W/ Engine'] = '\n'.join(list_vehicle)

# = = = = = = = = = = = = = = =

input_2.to_excel('./6.vehicle.xlsx', index=False)

input_2

Progress: 100%|███████████████████████████| 251/251 [00:02<00:00, 117.34it/s]


,Item Number,Vehicle W/o Engine,Vehicle W/ Engine
0,192731409825,Chevrolet Silverado 1500 1999-2013\nChevrolet ...,Chevrolet Silverado 1500 2010-2013 4.8L 294Cu....
1,192731591032,Dodge Ram 1500 2002-2008\nDodge Ram 2500 2003-...,Dodge Ram 1500 2004-2008 4.7L 287Cu. In. V8 FL...
2,192732802929,Buick Century 2000-2005\nBuick Regal 2000-2004...,Buick Century 2000-2005 3.1L 189Cu. In. V6 GAS...
3,192732835083,Chevrolet Blazer 1996-2005\nChevrolet S10 1996...,Chevrolet Blazer 1996-2005 4.3L 262Cu. In. V6 ...
4,192733013576,BMW Z4 2003-2008\nBMW 325Ci 2001-2006\nBMW 330...,BMW Z4 2003-2008 2.5L 2494CC 152Cu. In. l6 GAS...
...,...,...,...
246,196495713100,Ford Mustang 2015-2020,Ford Mustang 2015-2020 2.3L 2300CC 140Cu. In. ...
247,196495781333,Chevrolet Tahoe 2017-2021\nChevrolet Suburban ...,Chevrolet Tahoe 2017-2021 5.3L 5328CC 325Cu. I...
248,196497159761,Toyota Highlander 2001-2007,Toyota Highlander 2005-2007 3.3L 3300CC 202Cu....
249,196497166909,Ford Ranger 1985-1994\nMazda B2300 1994,Ford Ranger 1985-1994 2.3L 140Cu. In. l4 GAS S...
